In [1]:
from Lib.data_loader import DataLoader
from Lib.resnet_model import Resnet3DBuilder
from Lib.HistoryGraph import HistoryGraph
import Lib.image as img
from Lib.utils import mkdirs
import os




In [2]:
from math import ceil

In [3]:
from keras.optimizers import SGD

In [4]:
from keras.callbacks import ModelCheckpoint


In [5]:
target_size = (64,95)
nb_frames = 10
skip = 1
nb_classes = 27
#reg_factor = 0.0001     
batch_size = 64
input_shape = (nb_frames, ) + target_size + (3,)


In [6]:
workers = 8
use_multiprocessing = False
max_queue_size = 20


In [7]:
data_root = r"C:\Users\USER\Documents\Dataset"
data_vid = r"C:\Users\USER\Documents\Dataset\videos"
csv_labels = r"C:\Users\USER\Documents\Dataset\jester-v1-labels.csv"
csv_train = r"C:\Users\USER\Documents\Dataset\jester-v1-train.csv"
csv_val = r"C:\Users\USER\Documents\Dataset\jester-v1-validation.csv"
csv_test = r"C:\Users\USER\Documents\Dataset\jester-v1-test.csv"
model_name = 'resnet_3d_model'
data_model = r"C:\Users\USER\Documents\HandGestureRecognitionSystemProject\model"


In [8]:
path_model = os.path.join(data_root, data_model, model_name)
path_vid = os.path.join(data_root, data_vid)
path_labels = os.path.join(data_root, csv_labels)
path_train = os.path.join(data_root, csv_train)
path_val = os.path.join(data_root, csv_val)
path_test = os.path.join(data_root, csv_test)


print(f"path_vid exists: {os.path.exists(path_vid)}")
print(f"path_labels exists: {os.path.exists(path_labels)}")
print(f"path_train exists: {os.path.exists(path_train)}")
print(f"path_val exists: {os.path.exists(path_val)}")
print(f"path_test exists: {os.path.exists(path_test)}")

path_vid exists: True
path_labels exists: True
path_train exists: True
path_val exists: True
path_test exists: True


In [9]:
# Create DataLoader instance
data = DataLoader(
    path_vid=path_vid,
    path_labels=path_labels,
    path_train=path_train,
    path_val=path_val,
    path_test=path_test
)
mkdirs(path_model, 0o755)
mkdirs(os.path.join(path_model,  "graphs"), 0o755)




In [10]:
gen = img.ImageDataGenerator()
if hasattr(data, 'train_df'):
    gen_train = gen.flow_video_from_dataframe(
        data.train_df,
        path_vid,
        path_classes=path_labels,
        x_col='video_id',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        nb_frames=nb_frames,
        skip=skip,
        has_ext=True
    )
else:
    print("train_df not found in DataLoader")

if hasattr(data, 'val_df'):
    gen_val = gen.flow_video_from_dataframe(
        data.val_df,
        path_vid,
        path_classes=path_labels,
        x_col='video_id',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        nb_frames=nb_frames,
        skip=skip,
        has_ext=True
    )
else:
    print("val_df not found in DataLoader")
'''
gen_train = gen.flow_video_from_dataframe(data.train_df, path_vid, path_classes=path_labels, x_col='video id', y_col='labels', target_size=target_size, batch_size=batch_size, nb_frames=nb_frames, skip=skip, has_ext=True)
gen_val = gen.flow_video_from_dataframe(data.val_df, path_vid, path_classes=path_labels, x_col='video id', y_col='labels', target_size=target_size, batch_size=batch_size, nb_frames=nb_frames, skip=skip, has_ext=True)
'''

Found 118562 video folders belonging to 27 classes.
Found 14787 video folders belonging to 27 classes.


"\ngen_train = gen.flow_video_from_dataframe(data.train_df, path_vid, path_classes=path_labels, x_col='video id', y_col='labels', target_size=target_size, batch_size=batch_size, nb_frames=nb_frames, skip=skip, has_ext=True)\ngen_val = gen.flow_video_from_dataframe(data.val_df, path_vid, path_classes=path_labels, x_col='video id', y_col='labels', target_size=target_size, batch_size=batch_size, nb_frames=nb_frames, skip=skip, has_ext=True)\n"

In [11]:
from tensorflow.keras.optimizers import SGD
import os

input_shape = (64, 64, 64, 1)  # Example input shape (modify as needed)
nb_classes = 10                # Example number of classes (modify as needed)
reg_factor = 0.0001            # L2 regularization factor

# Build the ResNet 101 model with the correct arguments
resnet_model = Resnet3DBuilder.build_resnet_101(input_shape, nb_classes, reg_factor, drop_rate=0.5)

# Compile the model with an optimizer
optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=0.0301, nesterov=False)
resnet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Define the path where the model will be saved
path_model = './models'  # Example path (modify as needed)
model_file = os.path.join(path_model, 'resnetmodel.hdf5')

# Now you can proceed with saving the model or training it as needed

'''
resnet_model = Resnet3DBuilder.build_resnet_101(input_shape, nb_classes, reg_factor, drop_rate = 0.5)
optimizer = SGD(lr=0.01, momentum=0.9, decay=0.0301, nesterov=False)
resnet_model.compile(optimizer = optimizer, loss = "categorical_crossentropy" , metrics=["accuracy"])
model_file = os.path.join(path_model, 'resnetmodel.hdf5')
'''

'\nresnet_model = Resnet3DBuilder.build_resnet_101(input_shape, nb_classes, reg_factor, drop_rate = 0.5)\noptimizer = SGD(lr=0.01, momentum=0.9, decay=0.0301, nesterov=False)\nresnet_model.compile(optimizer = optimizer, loss = "categorical_crossentropy" , metrics=["accuracy"])\nmodel_file = os.path.join(path_model, \'resnetmodel.hdf5\')\n'

In [12]:
model_checkpointer = ModelCheckpoint(model_file, monitor='val_acc' , verbose=1, save_best_only=True, mode= 'max')


In [13]:
history_graph = HistoryGraph(model_path_name = os.path.join(path_model, "graphs"))

In [14]:
nb_sample_train = data.train_df["video_id"].size
nb_sample_val = data.val_df["video_id"].size



In [15]:
resnet_model.fit(
        gen_train,
        steps_per_epoch=ceil(nb_sample_train/batch_size),
        epochs=100,
        validation_data=gen_val,
        validation_steps=30,
        shuffle=True,
        verbose=1,
        workers=workers,
        max_queue_size=max_queue_size,
        use_multiprocessing=use_multiprocessing,
        callbacks=[model_checkpointer, history_graph])



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\USER\\Documents\\Dataset\\videos\\121656;Pushing Two Fingers Away'